In [2]:
import os, sys
import pandas as pd

In [5]:
def queryFiles(pd, startDate, endDate, filter_=None, columns=None):
    '''Function to get data from large csv files located in data/
    Filename to query: data.YYYYMMDD.gz
    Params: startDate, endDate (YYYYMMDD)
            filter (Example: filter = lambda df: df['EXCHANGE'] == 'HKSE')
    '''
    dateList = pd.date_range(start=pd.to_datetime(startDate, format='%Y%m%d'), end=pd.to_datetime(endDate, format='%Y%m%d'))
    dfs = list()
    for dt in dateList.tolist():
        file = 'data/data.{0:%Y%m%d}.gz'.format(dt)
        if os.path.exists(file):
            print('Parsing file: {}'.format(file))
            iter_csv = pd.read_csv(file, sep='|', compression='gzip', iterator=True, chunksize=100000)
            if filter_ == None:
                if columns == None:
                    df = pd.concat([chunk for chunk in iter_csv])
                else:
                    df = pd.concat([chunk[columns] for chunk in iter_csv])
            else:
                if columns == None:
                    df = pd.concat([chunk[filter_(chunk)] for chunk in iter_csv])
                else:
                    df = pd.concat([chunk[filter_(chunk)][columns] for chunk in iter_csv])
            dfs.append(df)
        else:
            print('Warning: file {} doesn\'t exist!'.format(file))
    dfAll = pd.concat(dfs)
    return dfAll